In [8]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres
from exputils import *

# Load Data

In [9]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

In [10]:
pairs = pd.read_csv('selected-pairs.tsv', header=None)[0].tolist()
train_dict = {pair: read_train(pair, dropbox_dir) for pair in pairs}
test_dict = {pair: read_test(pair, dropbox_dir) for pair in pairs}

# Experiment setup

In [11]:
window_sizes = [4, 6, 8, 10, 12, 24, 30, 48]
loops = len(pairs) * len(window_sizes)
loops

152

# Calc Supports

In [12]:
def get_supports(data, window_size):
    supports, _ = supres(data['ohlc4'].values, window_size)
    supports = pd.DataFrame(supports, columns=['index', 'support'])
    support_vals = supports['support'].values
    size = support_vals.shape[0]
    _return = (support_vals[1:] / support_vals[:size-1]) - 1
    _return = np.insert(_return, 0, 0)
    supports['return1'] = _return
    return supports

In [45]:
from itertools import product

def supports_f(args):
    data_dict, pair, window_size = args
    return pair, window_size, get_supports(data_dict[pair], window_size)

def calc_supports(data_dict, output_file):
    args = product([data_dict], pairs, window_sizes)
    results = run_parallel(supports_f, args)
    supports_dict = dict_from_list(results)
    save_model(supports_dict, output_file)

In [23]:
calc_supports(train_dict, 'supports_train.pkl')

In [46]:
calc_supports(test_dict, 'supports_test.pkl')

# Supports for ohlc data

In [33]:
from itertools import product

def find_support(i, supports):
    selected = supports[supports['index'] <= i]['support']
    return None if len(selected) == 0 else selected.iloc[-1]


def ohlc_supports_f(args):
    data_dict, supports_dict, pair, window_size = args
    supports = supports_dict[pair][window_size]
    data = data_dict[pair]
    s = [find_support(i, supports) for i in range(len(data))]
    return pair, window_size, s


def calc_ohlc_supports(data_dict, supports_dict, output_file):
    args = product([data_dict], [supports_dict], pairs, window_sizes)
    results = run_parallel(ohlc_supports_f, args)
    supports_for_data_dict = dict_from_list(results)
    save_model(supports_for_data_dict, output_file)

In [34]:
supports_dict_train = load_model('supports_train.pkl')
calc_ohlc_supports(train_dict, supports_dict_train, 'ohlc_supports_train.pkl')

In [50]:
supports_dict_test = load_model('supports_test.pkl')
calc_ohlc_supports(test_dict, supports_dict_test, 'ohlc_supports_test.pkl')